### Scraping the link

In [ ]:
import undetected_chromedriver as uc
import random

def random_user_agent():
    # This can be expanded. Use a library or list to randomly select a user-agent.
    user_agents = [
        "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/95.0.4638.54 Safari/537.36",
        # ... add more user-agents
    ]
    return random.choice(user_agents)

def random_delay(minimum=5, maximum=15):
    return time.sleep(random.uniform(minimum, maximum))

options = uc.ChromeOptions()
options.add_argument(f"user-agent={random_user_agent()}")

# Add other options like proxies here, if you have them

browser = uc.Chrome(options=options)

base_url = "https://www.realtor.com/apartments/Las-Vegas_NV/pg-10"


property_links = []

browser.get(url)

    # Introduce random delay after loading each page
random_delay()

    # Scroll through the page to load more content
scroll_page()

    # Wait until the presence of the property links
wait = WebDriverWait(browser, 60)  # Increased the wait time to 60 seconds
wait.until(EC.presence_of_all_elements_located((By.CSS_SELECTOR, "a.card-anchor")))

    # Extract the property links from the page
links = browser.find_elements(By.CSS_SELECTOR, "a.card-anchor")
for link in links:
    property_links.append(link.get_attribute("href"))

    # Introduce random delay before moving to the next page
random_delay()

# Close the browser
browser.quit()

# Save links to CSV
with open('property_links.csv', 'w', newline='', encoding='utf-8') as file:
    writer = csv.writer(file)
    writer.writerow(['Link'])
    for link in property_links:
        writer.writerow([link])

print("Scraping completed and saved to property_links.csv!")


### Scraping each links information

In [ ]:
import requests
from bs4 import BeautifulSoup
import csv
import re

# List of links you want to scrape
links = [
'https://www.realtor.com/rentals/details/3501-S-Maryland-Pkwy_Las-Vegas_NV_89169_M14058-58461',
'https://www.realtor.com/rentals/details/4000-E-Bonanza-Rd_Las-Vegas_NV_89110_M27725-71219',
'https://www.realtor.com/rentals/details/2120-Van-Patten-Pl_Las-Vegas_NV_89104_M19992-31808',
'https://www.realtor.com/rentals/details/9490-Bermuda-Rd_Las-Vegas_NV_89123_M14284-51408',
'https://www.realtor.com/rentals/details/3896-University-Center-Dr_Las-Vegas_NV_89119_M95695-89840',
'https://www.realtor.com/rentals/details/2255-Spruce-Goose-St_Las-Vegas_NV_89135_M97751-35398',
'https://www.realtor.com/rentals/details/9950-W-Tropicana-Ave_Las-Vegas_NV_89147_M97868-55464',
'https://www.realtor.com/rentals/details/5091-Moose-Falls-Dr_Las-Vegas_NV_89141_M29539-47285',
'https://www.realtor.com/rentals/details/6978-Walden-Park-St_Las-Vegas_NV_89166_M17335-05214',
'https://www.realtor.com/rentals/details/7565-Norman-Rockwell-Ln-Ste-110_Las-Vegas_NV_89143_M95950-01113',
'https://www.realtor.com/rentals/details/11055-Las-Vegas-Blvd-S_Las-Vegas_NV_89183_M90939-36612',
'https://www.realtor.com/rentals/details/7659-Glorietta-Bay-Ct_Las-Vegas_NV_89139_M19296-60665',
'https://www.realtor.com/rentals/details/3400-Cabana-Dr-15_Las-Vegas_NV_89122_M94543-50448',
'https://www.realtor.com/rentals/details/3024-Reef-View-St_Las-Vegas_NV_89117_M25843-96157',
'https://www.realtor.com/rentals/details/9156-Conquest-Ct_Las-Vegas_NV_89149_M22396-41923',
'https://www.realtor.com/rentals/details/5164-Mineral-Lake-Dr_Las-Vegas_NV_89122_M21178-23409',
'https://www.realtor.com/rentals/details/3750-Corpolo-Ave_Las-Vegas_NV_89141_M23958-51519',
'https://www.realtor.com/rentals/details/7209-Beachwood-Crest-St_Las-Vegas_NV_89166_M25908-79728',
'https://www.realtor.com/rentals/details/9428-Fort-Connah-Ct_Las-Vegas_NV_89178_M28705-83371',
'https://www.realtor.com/rentals/details/10617-Capitol-Peak-Ave_Las-Vegas_NV_89166_M10535-47525',
'https://www.realtor.com/rentals/details/8262-Vanity-Fair-Ln_Las-Vegas_NV_89113_M22681-13345',
'https://www.realtor.com/rentals/details/4715-Longshot-Dr_Las-Vegas_NV_89122_M17389-80283',
'https://www.realtor.com/rentals/details/7941-Violet-Dawn-St_Las-Vegas_NV_89149_M26442-87048',
'https://www.realtor.com/rentals/details/7105-Powderhorn-Cir_Las-Vegas_NV_89128_M28344-00770',
'https://www.realtor.com/rentals/details/6460-Hamilton-Grove-Ave_Las-Vegas_NV_89122_M24350-90418',
'https://www.realtor.com/rentals/details/5033-Whistling-Acres-Ave_Las-Vegas_NV_89131_M11631-14748',
'https://www.realtor.com/rentals/details/7912-Point-Oaks-Ct_Las-Vegas_NV_89149_M24793-02712',
'https://www.realtor.com/rentals/details/4792-W-Shelbourne-Ave_Las-Vegas_NV_89139_M13639-06795',
'https://www.realtor.com/rentals/details/7112-Mercurio-Ave_Las-Vegas_NV_89131_M25641-73407',
'https://www.realtor.com/rentals/details/7315-Larix-Rd_Las-Vegas_NV_89113_M29691-85907',
'https://www.realtor.com/rentals/details/8261-Oliver-Twist-Ln_Las-Vegas_NV_89113_M10510-50707',
'https://www.realtor.com/rentals/details/10525-Autumn-Pine-Ave-Apt-205_Las-Vegas_NV_89144_M13423-92454',
'https://www.realtor.com/rentals/details/8819-W-Oquendo-Rd_Las-Vegas_NV_89148_M10494-27730',
'https://www.realtor.com/rentals/details/595-S-Royal-Crest-Cir-Unit-7_Las-Vegas_NV_89169_M17649-29455',
'https://www.realtor.com/rentals/details/9594-Vega-Carpio-Ave_Las-Vegas_NV_89178_M28291-64246',
'https://www.realtor.com/rentals/details/5894-Grechetto-Ct_Las-Vegas_NV_89141_M26091-82381',
'https://www.realtor.com/rentals/details/8920-Picket-Fence-Ave_Las-Vegas_NV_89143_M29292-10403',
'https://www.realtor.com/rentals/details/4973-Blue-Marlin-Ave_Las-Vegas_NV_89115_M25339-22847',
'https://www.realtor.com/rentals/details/5725-Smoke-Ranch-Rd-Unit-C_Las-Vegas_NV_89108_M16486-35746',
'https://www.realtor.com/rentals/details/7224-Cottonsparrow-St_Las-Vegas_NV_89131_M15670-54301',
'https://www.realtor.com/rentals/details/725-S-Hualapai-Way_Las-Vegas_NV_89145_M10327-15621',
'https://www.realtor.com/rentals/details/3240-Las-Vegas-Blvd-N_Las-Vegas_NV_89115_M15790-49379',
'https://www.realtor.com/rentals/details/2700-N-Rainbow-Blvd_Las-Vegas_NV_89108_M12346-64196',
'https://www.realtor.com/rentals/details/9625-W-Russell-Rd_Las-Vegas_NV_89148_M23144-53806',
'https://www.realtor.com/rentals/details/818-E-Flamingo-Rd_Las-Vegas_NV_89119_M16729-90948',
'https://www.realtor.com/rentals/details/2375-Spruce-Goose-St_Las-Vegas_NV_89135_M23574-93064',
'https://www.realtor.com/rentals/details/6614-Blue-Diamond-Rd_Las-Vegas_NV_89139_M98789-01931',
'https://www.realtor.com/rentals/details/10136-Serenity-Star-Way_Las-Vegas_NV_89147_M94416-26958',
'https://www.realtor.com/rentals/details/9501-W-Sahara-Ave_Las-Vegas_NV_89117_M10102-07234',
'https://www.realtor.com/rentals/details/4450-Karen-Ave_Las-Vegas_NV_89121_M29403-05227',

]

headers = {
    "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36"
}

data_to_save = []

for link in links:
    response = requests.get(link, headers=headers)
    
    if response.status_code != 200:
        print(f"Failed to retrieve the webpage: {link}")
        continue

    # Parse page with BeautifulSoup
    soup = BeautifulSoup(response.content, 'html.parser')
    
    # Extract apartment name
    name_tag = soup.find('h1', {'class': lambda x: x and 'base__StyledType-rui' in x and 'name' in x})
    apartment_name = name_tag.text if name_tag else None
    
    # Extract apartment address
    address_tag = soup.find('p', {'class': lambda x: x and 'base__StyledType-rui' in x and 'address' in x})
    apartment_address = address_tag.text.strip() if address_tag else None

    # Extract ZIP code
    zip_code = None
    if apartment_address:
        zip_code = apartment_address.split()[-1]
    
    # Extract price
    price_tag = soup.find('div', class_=re.compile('Pricestyles__StyledPrice-rui__'))
    price = price_tag.text.split('-')[0].strip().replace('$', '') if price_tag else None

    # Extract square feet
    sqft_tag = soup.find('li', class_=re.compile('PropertySqftMetastyles__StyledPropertySqftMeta-rui__'))
    sqft = sqft_tag.find('span', class_='meta-value').text.split('-')[0].strip().replace('sqft', '').strip() if sqft_tag and sqft_tag.find('span', class_='meta-value') else None

    # Extract beds
    beds_tag = soup.find('li', attrs={"data-testid": "property-meta-beds"})
    beds_value = beds_tag.find('span', attrs={"data-testid": "meta-value"}).text if beds_tag else None

    if beds_value:
        if 'Studio' in beds_value:
            beds = 'Studio'
        else:
            beds = beds_value.split('-')[0].strip()

    # Extract baths
    baths_tag = soup.find('li', attrs={"data-testid": "property-meta-baths"})
    baths_value = baths_tag.find('span', attrs={"data-testid": "meta-value"}).text if baths_tag else None
    baths = baths_value.split('-')[0].strip() if baths_value else None
    
    # Extract Property Type
    property_type = None
    type_tag = soup.find('div', string='Property type')
    if type_tag and type_tag.find_next_sibling():
        property_type = type_tag.find_next_sibling().text

    # Extract Year Built
    year_built = None
    year_built_tag = soup.find('div', string='Year built')
    if year_built_tag and year_built_tag.find_next_sibling():
        year_built = year_built_tag.find_next_sibling().text
    
    # Append data to the list
    data_to_save.append([link, apartment_name, apartment_address, zip_code, price, sqft, beds, baths, property_type, year_built])


# Write data to CSV
with open('apartment_data56.csv', 'w', newline='', encoding='utf-8') as csvfile:
    writer = csv.writer(csvfile)
    # Added 'URL' to the header
    writer.writerow(['URL', 'Apartment Name', 'Apartment Address', 'ZIP Code', 'Price', 'Sqft', 'Beds', 'Baths', 'Property Type', 'Year Built'])  
    writer.writerows(data_to_save)

print("Data saved to 'apartment_data.csv'")
